In [1]:
import datasets
from tokenizers import TRIETokenizerFast
from matplotlib import pyplot as plt
import json
from tqdm.notebook import tqdm

In [ ]:
tiny_textbook = datasets.load_dataset('nampdn-ai/tiny-textbooks', cache_dir='./corpus')['train']

In [ ]:
with open('tiny-textbooks.jsonl', 'w') as file:
    for i, e in enumerate(tiny_textbook):
        if i > 0:
            file.write('\n')
        file.write(json.dumps({'text': e['textbook']}, ensure_ascii=False))

In [ ]:
mini_pile = datasets.load_dataset('JeanKaddour/minipile', cache_dir='./corpus')['train']

In [ ]:
with open('corpus/minipile.jsonl', 'w') as file:
    for i, e in tqdm(enumerate(mini_pile), total=len(mini_pile)):
        if i > 0:
            file.write('\n')
        file.write(json.dumps({'text': e['text']}, ensure_ascii=False))

In [5]:
en_wiki_100k = datasets.load_dataset('teven/enwiki_100k', cache_dir='./corpus')['train']
en_wiki_100k_filtered = en_wiki_100k.filter(lambda i:i['text'].endswith('.'))

In [7]:
with open('corpus/enwiki_100k_filtered.jsonl', 'w') as file:
    for i, e in tqdm(enumerate(en_wiki_100k_filtered), total=len(en_wiki_100k_filtered)):
        if i > 0:
            file.write('\n')
        file.write(json.dumps({'text': e['text']}, ensure_ascii=False))

  0%|          | 0/532383 [00:00<?, ?it/s]